In [12]:
import gradio as gr
from transformers import pipeline
import numpy as np
import librosa
import time

#transcriber = pipeline("automatic-speech-recognition", model="asapp/sew-d-tiny-100k-ft-ls100h")
transcriber = pipeline(
    task="zero-shot-audio-classification", model="laion/clap-htsat-unfused"
)
candidate_labels = ["Sound of a hand hitting a desk", "Sound of a clap", "Sound of random noise"]

def transcribe(audio):
    start=time.time()
    sr, y = audio
    print(y.shape)
    print(np.max(np.abs(y)))
    y = y.astype(np.float32)
    if len(y.shape)>=2: y = np.mean(y, axis=1)
    y /= np.max(np.abs(y))
    return transcriber(y, candidate_labels=candidate_labels)


demo = gr.Interface(
    transcribe,
    gr.Audio(sources=["microphone"]),
    "text",
)

demo.launch()


Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


(139952,)
32767


ERROR:    Exception in ASGI application
  + Exception Group Traceback (most recent call last):
  |   File "c:\Users\angel\anaconda3\envs\AIM-S2024-AudioGaming\Lib\site-packages\starlette\middleware\base.py", line 192, in __call__
  |     await response(scope, wrapped_receive, send)
  |   File "c:\Users\angel\anaconda3\envs\AIM-S2024-AudioGaming\Lib\site-packages\starlette\responses.py", line 257, in __call__
  |     async with anyio.create_task_group() as task_group:
  |   File "c:\Users\angel\anaconda3\envs\AIM-S2024-AudioGaming\Lib\site-packages\anyio\_backends\_asyncio.py", line 678, in __aexit__
  |     raise BaseExceptionGroup(
  | ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "c:\Users\angel\anaconda3\envs\AIM-S2024-AudioGaming\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 408, in run_asgi
    |     result = await app(  # type: ignore[func-returns-value

(157592,)
31063


In [13]:
import gradio as gr
from transformers import pipeline
import numpy as np

transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(stream, new_chunk):
    sampling_rate, y = new_chunk
    if len(y.shape) > 1: y = np.mean(y,axis=1)
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    if stream is not None:
        stream = np.concatenate([stream, y])
    else:
        stream = y
    return stream, transcriber({"sampling_rate": sampling_rate, "raw": stream})["text"]


demo = gr.Interface(
    transcribe,
    ["state", gr.Audio(sources=["microphone"], streaming=True)],
    ["state", "text"],
    live=True,
)

demo.launch()


Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.
